### 1. ElasticSearch 연동 및 확인

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

In [2]:
es = Elasticsearch('http://13.209.84.139:9200')
es.info()

{'name': 'Wilshere_Elasticsearch',
 'cluster_name': 'Wilshere_ES',
 'cluster_uuid': 'e8ZntsY7R7CEZnj8Odv-Rw',
 'version': {'number': '7.12.1',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': '3186837139b9c6b6d23c3200870651f10d3343b7',
  'build_date': '2021-04-20T20:56:39.040728659Z',
  'build_snapshot': False,
  'lucene_version': '8.8.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

### 2. ElasticSearch 에 index 생성

In [3]:
# 데이터를 색인화하기 위한 함수
def indexing(es, index_name):
    # 이미 존재할 경우 삭제하고 다시 만들기
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)

    # 인덱스 생성
    print(es.indices.create(index=index_name))

In [11]:
# 인덱스명을 정하기 (자유롭게)
index_name = 'en_wiki_5'
#index_name = 'temp1'
indexing(es, index_name)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'en_wiki_5'}


In [ ]:
en_wiki_1
en_wiki_2
en_wiki_3
en_wiki_4
en_wiki_5

### 3. ElasticSearch 에 데이터 적재(인덱싱)

In [ ]:
./en_wiki_dump/enwiki-20210520_1_result/AA~AD || AA/AB/AC/AD
./en_wiki_dump/enwiki-20210520_2_result/AA~AF || AA/AB/AC/AD/AE/AF
./en_wiki_dump/enwiki-20210520_3_result/AA~AE || AA/AB/AC/AD/AE
./en_wiki_dump/enwiki-20210520_4_result/AA~AF || AA/AB/AC/AD/AE/AF
./en_wiki_dump/enwiki-20210520_5_result/AA~AF || AA/AB/AC/AD/AE/AF
여기까지

In [17]:
import os
import xml.etree.ElementTree as ET
import json, codecs
import xmltodict
from collections import OrderedDict

path = './en_wiki_dump/enwiki-20210520_5_result/AF/'
file_list = os.listdir(path)

for i in range(len(file_list)):
    fr = open(path + file_list[i],'r', encoding="utf-8")
    data = fr.readlines()
    
    for j in range(len(data)):
        es.index(index=index_name, doc_type='string', body=data[j])

    fr.close()
es.indices.refresh(index=index_name)

{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}

### 4. ElasticSearch 에 적재된 데이터 검색

In [35]:
results = es.search(index=index_name, body={'from':0, 'size':5, 'query':{'match':{'text':'computer'}}})
for result in results['hits']['hits']:
    print('score:', result['_score'], 'source:', result['_source'])

score: 5.802776 source: {'id': '174397', 'revid': '41419081', 'url': 'https://en.wikipedia.org/wiki?curid=174397', 'title': 'Computer addiction', 'text': 'Computer addiction is a form of behavioral addiction that can be described as the excessive or compulsive use of the computer, which persists despite serious negative consequences for personal, social, or occupational function. Another clear conceptualization is made by Block, who stated that "Conceptually, the diagnosis is a compulsive-impulsive spectrum disorder that involves online and/or offline computer usage and consists of at least three subtypes: excessive gaming, sexual preoccupations, and e-mail/text messaging". Computer addiction is not currently included in the "Diagnostic and Statistical Manual of Mental Disorders (DSM-5)" as an official disorder. The concept of computer addiction is broadly divided into two types, namely offline computer addiction, and online computer addiction. Offline computer addiction is normally us